<a href="https://colab.research.google.com/github/Lameeselbakr55/NTI_AI_Projects/blob/master/Other%20Dataset%20Models/Keras_LSTM_Movie_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using Keras LSTM for movie Reviews Classifying


##Dataloading and preparation

**Keras for Beginners: Implementing a Recurrent Neural Network**

https://victorzhou.com/blog/keras-rnn-tutorial/

**The Problem: Classifying Movie Reviews**

In [ ]:
!wget https://victorzhou.com/movie-reviews-dataset.zip

--2020-11-24 18:02:07--  https://victorzhou.com/movie-reviews-dataset.zip
Resolving victorzhou.com (victorzhou.com)... 104.27.147.219, 104.27.146.219, 172.67.153.220, ...
Connecting to victorzhou.com (victorzhou.com)|104.27.147.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62951389 (60M) [application/zip]
Saving to: ‘movie-reviews-dataset.zip’

movie-reviews-datas 100%[===================>]  60.03M  11.1MB/s    in 6.3s    

2020-11-24 18:02:14 (9.59 MB/s) - ‘movie-reviews-dataset.zip’ saved [62951389/62951389]



In [ ]:
!unzip movie-reviews-dataset

Streaming output truncated to the last 5000 lines.
  inflating: __MACOSX/movie-reviews-dataset/train/pos/._1599_7.txt  
  inflating: movie-reviews-dataset/train/pos/2174_8.txt  
  inflating: __MACOSX/movie-reviews-dataset/train/pos/._2174_8.txt  
  inflating: movie-reviews-dataset/train/pos/2309_9.txt  
  inflating: __MACOSX/movie-reviews-dataset/train/pos/._2309_9.txt  
  inflating: movie-reviews-dataset/train/pos/12034_10.txt  
  inflating: __MACOSX/movie-reviews-dataset/train/pos/._12034_10.txt  
  inflating: movie-reviews-dataset/train/pos/11703_9.txt  
  inflating: __MACOSX/movie-reviews-dataset/train/pos/._11703_9.txt  
  inflating: movie-reviews-dataset/train/pos/5619_9.txt  
  inflating: __MACOSX/movie-reviews-dataset/train/pos/._5619_9.txt  
  inflating: movie-reviews-dataset/train/pos/2928_10.txt  
  inflating: __MACOSX/movie-reviews-dataset/train/pos/._2928_10.txt  
  inflating: movie-reviews-dataset/train/pos/7096_10.txt  
  inflating: __MACOSX/movie-reviews-dataset/train/p

In [ ]:
import keras
from keras.models import *
from keras.layers import *
import cv2
import os
import numpy as np
import pandas as pd
#import keras as K
import tensorflow as tf
import matplotlib.pyplot as plt
from cv2 import resize
from tensorflow.keras.preprocessing import text_dataset_from_directory

In [ ]:



DATASET_DIR='/content/movie-reviews-dataset/train'
os.listdir(DATASET_DIR)


['pos', 'neg', '.DS_Store']

In [ ]:
# Assumes you're in the root level of the dataset directory.
# If you aren't, you'll need to change the relative paths here.
train_data = text_dataset_from_directory('/content/movie-reviews-dataset/train')
test_data = text_dataset_from_directory('/content/movie-reviews-dataset/test')

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace

def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

train_data = prepareData('/content/movie-reviews-dataset/train')
test_data = prepareData('/content/movie-reviews-dataset/test')

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0]) # 0 = negative, 1 = positive

b'Certainly one of the finest movies I have seen for quite some time. Exquisite direction and flawless acting make this a very entertaining and often moving film. Denzel Washington plays one of his most engaging and emotional roles to date, and the rest of the cast perform beautifully. Christopher Walken is of course superb in his part although he did not appear as often as I would have liked. A story of ultimate greed that backfires is offset against a childs innocence and love. This is also a film for action movie lovers as it has its fair share of bullets, rockets and revenge. The location of Mexico City adds a feel of seediness and corruption which in itself is an eye opener. All in all, a truly gripping film from beginning to end. Highly recommended!'
1


##Model Fitting

3. Building the Model


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input

model = Sequential()
model.add(Input(shape=(1,), dtype="string"))

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  # Max vocab size. Any words outside of the max_tokens most common ones
  # will be treated the same way: as "out of vocabulary" (OOV) tokens.
  max_tokens=max_tokens,
  # Output integer indices, one per string token
  output_mode="int",
  # Always pad or truncate to exactly this many tokens
  output_sequence_length=max_len,
)

In [ ]:
# Call adapt(), which fits the TextVectorization layer to our text dataset.
# This is when the max_tokens most common words (i.e. the vocabulary) are selected.
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

In [ ]:
from tensorflow.keras.layers import Embedding

# Previous layer: TextVectorization
max_tokens = 1000
# ...
model.add(vectorize_layer)

# Note that we're using max_tokens + 1 here, since there's an
# out-of-vocabulary (OOV) token that gets added to the vocab.
model.add(Embedding(max_tokens + 1, 128))

 The Recurrent Laye

In [ ]:
from tensorflow.keras.layers import LSTM

# 64 is the "units" parameter, which is the
# dimensionality of the output space.
model.add(LSTM(64))

In [ ]:
from tensorflow.keras.layers import Dense

model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

Compiling and training the Model

In [ ]:
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

In [ ]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 60s 77ms/step - loss: 0.5275 - accuracy: 0.7344
Epoch 2/10
782/782 [==============================] - 62s 80ms/step - loss: 0.4436 - accuracy: 0.7974
Epoch 3/10
782/782 [==============================] - 62s 79ms/step - loss: 0.4154 - accuracy: 0.8110
Epoch 4/10
782/782 [==============================] - 62s 79ms/step - loss: 0.3911 - accuracy: 0.8236
Epoch 5/10
782/782 [==============================] - 66s 85ms/step - loss: 0.3704 - accuracy: 0.8332
Epoch 6/10
782/782 [==============================] - 62s 80ms/step - loss: 0.3511 - accuracy: 0.8451
Epoch 7/10
782/782 [==============================] - 64s 82ms/step - loss: 0.3298 - accuracy: 0.8561
Epoch 8/10
782/782 [==============================] - 63s 80ms/step - loss: 0.3176 - accuracy: 0.8602
Epoch 9/10
782/782 [==============================] - 64s 82ms/step - loss: 0.3024 - accuracy: 0.8698
Epoch 10/10
782/782 [==============================] - 64s 82ms/step - loss: 0.291

##Results

In [ ]:
# Should print a very high score like 0.98.
print(model.predict([
  "i loved it! highly recommend it to anyone and everyone looking for a great movie to watch.",
]))

# Should print a very low score like 0.01.
print(model.predict([
  "this was awful! i hated it so much, nobody should watch this. the acting was terrible, the music was terrible, overall it was just bad.",
]))

[[0.9826149]]
[[0.00574061]]


In [ ]:
print(model.predict(["رثقغ لاشي",]))
print(model.predict([" verall it was just bad",]))
print(model.predict([" verall it was just good",]))

[[0.8565128]]
[[0.09227765]]
[[0.8577808]]
